# Least squares

In [1]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.linear_model import LinearRegression as LR

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = datasets.load_diabetes()
X_train, y_train = data.data, data.target

In [2]:
def fit_inverse(X, y):
    """Direct method using the inverse"""
    return np.linalg.inv(np.transpose(X) @ X) @ np.transpose(X) @ y

In [3]:
w = fit_inverse(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
assert error <= 1e-13
print("prediction error: ", error)

prediction error:  3.166933990047579e-14


In [4]:
def fit_cholesky(X, y):
    """ Cholesky approach """
    # Xh is the transpose of X
    Xh = np.transpose(X)
    L = scipy.linalg.cholesky(Xh @ X, lower=True)
    w1 = scipy.linalg.solve_triangular(L, Xh @ y, lower=True)
    return scipy.linalg.solve_triangular(np.transpose(L), w1)

In [5]:
w = fit_cholesky(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
assert error <= 1e-13
print("prediction error: ", error)

prediction error:  5.64667468031009e-14


In [14]:
def fit_qr(X, y):
    """ QR approach"""
    # Xh is the transpose of X
    Xh = np.transpose(X)
    Q, R = np.linalg.qr(Xh @ X)
    w1 = np.transpose(Q) @ Xh @ y
    return scipy.linalg.solve_triangular(R, w1)

In [15]:
w = fit_qr(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
assert error <= 1e-13
print("prediction error: ", error)

prediction error:  5.555259025460741e-14


In [16]:
def fit_svd(X, y):
    """ SVD approach"""
    # Xh is the transpose of X
    Xh = np.transpose(X)
    U, S, Vh = np.linalg.svd(Xh @ X)
    w1 = np.transpose(U) @ Xh @ y
    w2 = np.zeros(len(S))
    for i in range(len(S)):
        w2[i] = (1 / S[i]) * w1[i]
    return np.transpose(Vh) @ w2

In [17]:
w = fit_svd(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
assert error <= 1e-13
print("prediction error: ", error)

prediction error:  4.515320450796067e-14


## Combine everything in a class

In [18]:
class LinearRegression():
    def __init__(self, fit_intercept=True, method="inverse"):
        self.w = 0
        self.fit_intercept = fit_intercept # bias
        self.method = method
    
    def fit(self, X, y):
        ones = np.ones((len(X), 1))
        if self.fit_intercept:
            X = np.hstack((X, ones))
        if self.method == "inverse":
            self.w = fit_inverse(X, y)
        elif self.method == "cholesky":
            self.w = fit_cholesky(X, y)
        elif self.method == "qr":
            self.w = fit_qr(X, y)
        elif self.method == 'svd':
            self.w = fit_svd(X, y)
        return self.w
        
    def predict(self, X):
        ones = np.ones((len(X), 1))
        if self.fit_intercept:
            X = np.hstack((X, ones))  
        return X @ self.w
        

### without the bias term

In [19]:
# DIRECT INVERSE APPROACH
sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
assert error <= 1e-11
print("prediction error: ", error)

prediction error:  2.5088948915590246e-13


In [20]:
# OTHER APPROACHES
sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model_cholesky = LinearRegression(fit_intercept=False, method="cholesky")
model_cholesky.fit(X_train, y_train)
pred_cholesky = model_cholesky.predict(X_train)

error_cholesky = rel_error(pred_cholesky, sk_pred)
assert error_cholesky <= 1e-11
print("prediction error cholesky: ", error_cholesky)

model_qr = LinearRegression(fit_intercept=False, method="qr")
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
assert error_qr <= 1e-11
print("prediction error qr: ", error_qr)

model_svd = LinearRegression(fit_intercept=False, method="svd")
model_svd.fit(X_train, y_train)
pred_svd = model_cholesky.predict(X_train)

error_cholesky = rel_error(pred_svd, sk_pred)
assert error_cholesky <= 1e-11
print("prediction error svd: ", error_cholesky)

prediction error cholesky:  1.3061785085982075e-13
prediction error qr:  1.6198336625935292e-13
prediction error svd:  1.3061785085982075e-13


### with the bias term

In [21]:
# DIRECT INVERSE APPROACH
sk_model = LR(fit_intercept=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
assert error <= 1e-14
print("prediction error: ", error)

prediction error:  4.35488493420556e-15


In [22]:
# OTHER APPROACHES
sk_model = LR(fit_intercept=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model_cholesky = LinearRegression(fit_intercept=True, method="cholesky")
model_cholesky.fit(X_train, y_train)
pred_cholesky = model_cholesky.predict(X_train)

error_cholesky = rel_error(pred_cholesky, sk_pred)
assert error_cholesky <= 1e-11
print("prediction error cholesky: ", error_cholesky)

model_qr = LinearRegression(fit_intercept=True, method="qr")
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
assert error_qr <= 1e-11
print("prediction error qr: ", error_qr)

model_svd = LinearRegression(fit_intercept=True, method="svd")
model_svd.fit(X_train, y_train)
pred_svd = model_cholesky.predict(X_train)

error_cholesky = rel_error(pred_svd, sk_pred)
assert error_cholesky <= 1e-11
print("prediction error svd: ", error_cholesky)

prediction error cholesky:  3.9169122150176046e-15
prediction error qr:  3.2131831680749515e-15
prediction error svd:  3.9169122150176046e-15


Compare the running time of the different approaches on a large dataset